In [1]:
from gensim import models

# takes a lilbit
# model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  
model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  

In [40]:
# imports and permanent stuff

import requests
import json
import pickle
import os
import numpy as np
import time

list_of_scrape_times = []
list_of_types = ['text','log','contact','calender','file','tweets']

In [8]:
## ELEMENTWISE FEATURIZING FUNCTION DEFINITIONS

# takes in element of list_of_jsons and scrape date, spits out text frequency 
def textFreq(texts, scrapedate):
    
    a = texts.json()
    
    seconds_intwo_weeks = 1209600;
    
    two_weeks_prior = scrapedate - seconds_intwo_weeks
    
    text_date = 0
    saved_index = 0
    
    for i in range(0,len(a)):
        text_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
        if (two_weeks_prior < text_date):
            saved_index = len(a) - (i+1)
            break
    
    return float(saved_index+1)/14
    
    
# takes in element of list_of_jsons(resp object) and scrape date, spits out call frequency 
def callFreq(calls, scrapedate):
    
    a = calls.json()
    
    seconds_intwo_weeks = 1209600;
    
    two_weeks_prior = scrapedate - seconds_intwo_weeks
    
    call_date = 0
    saved_index = 0
    
    for i in range(0,len(a)):
        call_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
        if (two_weeks_prior < call_date):
            saved_index = len(a) - (i+1)
            break
    
    return float(saved_index+1)/14

In [44]:
r = requests.get('http://depressionmqp.wpi.edu:8080/getids')
list_of_idtime = r.json()
list_of_ids = []
list_of_times = []

for i in range(0,len(list_of_idtime)):
    list_of_ids.append( list_of_idtime[i]['id'].encode('ascii','ignore') )
    
for i in range(0,len(list_of_idtime)):
    list_of_times.append( list_of_idtime[i]['date'] )

#id1 = list_of_ids[0].encode('ascii','ignore')

number_cols = len(list_of_types)
number_rows = len(list_of_ids)
list_of_jsons = [[0] * number_cols for i in range(number_rows)]


### create directory for this particular scrape
timenow  = str(int(time.time())) # for temporal congruency
list_of_scrape_times.append(timenow)
os.mkdir('./datafor' + timenow)
filedir = './datafor' + timenow


for i in range(0,number_rows):
    for j in range(0,number_cols):
        temp = requests.get('http://depressionmqp.wpi.edu:8080/getdata?id=' + str(int(list_of_ids[0])) + '&type=' + list_of_types[j])
        
        pickle.dump(temp, open( filedir + "/DP" + str(int(list_of_ids[i])) +  list_of_types[j] + ".p", "wb" ))
        
        # loads it into memory. i will not use this for now
        # for architectural sanity
        # list_of_jsons[i][j] = temp

In [43]:
## DELETE DIRECTORY CELL

import shutil

def deletdatboi(nombre):
    shutil.rmtree('./datafor' + list_of_scrape_times[nombre])

#deletdatboi(1)

list_of_scrape_times


[]

In [12]:
num_of_people = len(list_of_ids)

featureVectorCF = np.zeros((num_of_people,1))
featureVectorTF = np.zeros((num_of_people,1))
featureVectorTW = np.zeros((num_of_people,300))


for i in range(0, num_of_people):
    
    list_of_jsons[i][1]
    
    a1 = pickle.load( open( filedir + "/DP" + str(int(list_of_ids[i])) +  "log" + ".p", "rb" )) 
    a2 = pickle.load( open( filedir + "/DP" + str(int(list_of_ids[i])) +  "text" + ".p", "rb" )) 
    a3 = pickle.load( open( filedir + "/DP" + str(int(list_of_ids[i])) +  "tweets" + ".p", "rb" )) 

    
    featureVectorCF[i] = callFreq(a1, list_of_times[i])
    featureVectorTF[i] = textFreq(a2, list_of_times[i])
    featureVectorTW[i] = 

    
featureMatrix = np.hstack((featureVectorCF,featureVectorTF))

#featureMatrix

In [126]:
## WORD2VEC BB

a3 = pickle.load( open( filedir + "/DP" + str(int(list_of_ids[0])) +  "tweets" + ".p", "rb" )) 
json.loads(a3.json()[0])['text']

# input is tweet, returns vector embedding of entire tweet.
# eg: a3.json()[0]
def tweetToEmbedding(tweet):
    
    q = json.loads(tweet)['text'].split()
    
    sumVector = np.zeros((300,))

    for i in range(0,len(q)):
        try:
            sumVector += model[q[i]]
        except KeyError:
            pass
    
    return sumVector


def embeddingToMastersum(tweets):
    
    masterSum = np.zeros((300,))
    
    for i in range(0,10):
        try:
            masterSum += tweetToEmbedding(tweets.json()[i])
        except KeyError:
            pass

    return masterSum


In [143]:
w = embeddingToMastersum(a3)
w.reshape((-1, 1)).T.shape


(1, 300)

In [125]:
q = json.loads(a3.json()[1])['text'].split()
#a3.json()[0]